## Homework 08:  Parallel Programming 01

## Due Date: Apr 12, 2021, 04:00pm

#### Firstname Lastname: Wenxin Zhang

#### E-mail: wz2164@nyu.edu

#### Enter your solutions and submit this notebook

---

**Problem 1 (50p)**

Write an MPI program `sol08pr01.py` that does the following for some arbitrary number of processes $N \geq 2$. Here the number of processes $N$ is given as `N` while calling the code `sol08pr01.py` as: 

`mpirun -n N python3 sol08pr01.py`


Every process will contain one buffer with one integer variable, each of which is initialized to $0$.

For $r=0, 1, \dots, N - 1$, Process $r$ updates its buffer to the value received by $r-1$ (this should only be done for $r \geq 1$), then it squares its rank $r$, adds the result $r^2$ to the value of its own buffer, and then sends the sum to Process $r + 1$. Note that for $r=N-1$ the result will be sent to Process $0$, i.e. by convention, Process $N$ is the same as Process $0$. At the end Process $0$ prints the received value. 

Provide results for: $N = 10$, $N = 15$, $N = 20$, $N = 25$. These are probably more than the available processes on your machine: you can use the option `--oversubscribe` in `mpirun` to let MPI run things anyway.



**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code. 



In [4]:
%%writefile sol08pr01.py
import numpy
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
N = comm.Get_size()

value = numpy.zeros(1) 

### For the Process 0
if rank == 0:
    print("Process", rank, "before receiving has the number", value[0])
    
    # Send the value to the Process 1
    comm.Send(value, dest=1) 
    print("Process", rank, "drew the number", value[0])
    
    # Receive the value from the Process N-1
    comm.Recv(value, source=N-1)
    print("Process", rank, "received the number", value[0], "from process ", N-1)
    
    
### For the Process N-1
elif rank == N-1:
    print("Process", rank, "before receiving has the number", value[0])
    
    # Receive the value from the Process N-2
    comm.Recv(value, source=N-2)
    print("Process", rank, "received the number", value[0])  
    
    # Add r^2 to the value 
    value = value + rank**2
    
    # Send the updated value to the Process 0
    comm.Send(value, dest=0) 

    
### For the Process rank_i, where i belongs to (1, 2, 3, ... N-2)
else:
    print("Process", rank, "before receiving has the number", value[0])
    
    # Receive the value from the Process rank-1
    comm.Recv(value, source=rank-1)
    print("Process", rank, "received the number", value[0])    

    # Add r^2 to the value 
    value = value + rank**2
    
    # Send the updated value to the Process rank+1
    comm.Send(value, dest=rank+1) 

Overwriting sol08pr01.py


In [7]:
# !mpirun -n 10 -oversubscribe python3 sol08pr01.py

zsh:1: command not found: mpiexec


In [6]:
# !mpirun -n 15 python3 sol08pr01.py

zsh:1: command not found: mpirun


In [9]:
!mpirun -n 20 python3 sol08pr01.py

Process 1 before receiving has the number 0.0
Process 1 received the number 0.0
Process 2 before receiving has the number 0.0
Process 2 received the number 1.0
Process 3 before receiving has the number 0.0
Process 3 received the number 5.0
Process 4 before receiving has the number 0.0
Process 4 received the number 14.0
Process 5 before receiving has the number 0.0
Process 5 received the number 30.0
Process 6 before receiving has the number 0.0
Process 6 received the number 55.0
Process 7 before receiving has the number 0.0
Process 7 received the number 91.0
Process 8 before receiving has the number 0.0
Process 8 received the number 140.0
Process 9 before receiving has the number 0.0
Process 9 received the number 204.0
Process 10 before receiving has the number 0.0
Process 10 received the number 285.0
Process 11 before receiving has the number 0.0
Process 11 received the number 385.0
Process 12 before receiving has the number 0.0
Process 12 received the number 506.0
Process 13 before re

In [10]:
!mpirun -n 25 python3 sol08pr01.py

Process 1 before receiving has the number 0.0
Process 1 received the number 0.0
Process 2 before receiving has the number 0.0
Process 2 received the number 1.0
Process 3 before receiving has the number 0.0
Process 3 received the number 5.0
Process 4 before receiving has the number 0.0
Process 4 received the number 14.0
Process 5 before receiving has the number 0.0
Process 5 received the number 30.0
Process 6 before receiving has the number 0.0
Process 6 received the number 55.0
Process 7 before receiving has the number 0.0
Process 7 received the number 91.0
Process 8 before receiving has the number 0.0
Process 8 received the number 140.0
Process 9 before receiving has the number 0.0
Process 9 received the number 204.0
Process 10 before receiving has the number 0.0
Process 10 received the number 285.0
Process 11 before receiving has the number 0.0
Process 11 received the number 385.0
Process 12 before receiving has the number 0.0
Process 12 received the number 506.0
Process 13 before re

---

**Problem 2 (50p)**

Write an MPI program that does the following. There are two processes 0 and 1 that have to exchange $T=10$ messages.  


Process 0 initially reads two float variables from the standard input, call them $x, y$, and must ensure $x \neq 0$ and $y \neq 0$. For example this can be done as:

```
import sys


for line in sys.stdin:
    x = float(line)        
    if x != 0.0:
        break
for line in sys.stdin:
    y = float(line)        
    if y != 0.0:
        break
```


Both Process 0 and Process 1 will carry main results in an element that is part of a process buffer and called $p$. The value in $p$ is initially set to $1$. 


Now the exchange of messages is as follows.


0. Message00: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

1. Message01: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

2. Message01: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

3. Message02: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.


etc.

8. Message08: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

9. Message09: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

Finally, Process 0 prints the value in $p$ as a final result. 


Write the code that implements the protocol above. Additionally, provide results for: $(x, y) = (2, 4)$, $(x, y) = (1, 3)$, $(x, y) = (5, 7)$ and $(x, y) = (5, 10)$.


**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code.



In [11]:
%%writefile sol08pr02.py
import sys
import numpy as np
from mpi4py import MPI

x = float(sys.argv[1])
y = float(sys.argv[2])
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

p_init = 1.0
element= np.array([p_init, x, y])

### Regard Process0->Process1->Process0 as a loop
for i in range(5):
    if rank == 0:
        # Multiply p by x
        element[0] = element[0] * element[1]
        
        # Send the updated value of p to Process 1 with tag=0
        comm.Send(element, dest=1, tag=0)
        
        # Receive the updated value from Process 1 with tag=1
        comm.Recv(element, source=1, tag=1)
     
        # Get the output when i = 4
        if i == 4:
            print("Final p:", element[0])

    else:
        # Receive the updated value from Process 0 with tag=0
        comm.Recv(element, source=0, tag=0)
        
        # Divide p by y
        element[0] = float(element[0] / element[2])
        
        # Send the updated value of p to Process 0 with tag=1
        comm.Send(element, dest=0, tag=1)

Writing sol08pr02.py


In [12]:
!mpirun -n 2 python3 sol08pr02.py 2 4

Final p: 0.03125


In [13]:
!mpirun -n 2 python3 sol08pr02.py 1 3

Final p: 0.004115226337448559


In [14]:
!mpirun -n 2 python3 sol08pr02.py 5 7 

Final p: 0.1859344320818706


In [15]:
!mpirun -n 2 python3 sol08pr02.py 5 10

Final p: 0.03125
